In [1]:
import trimesh
import numpy as np
import copy
import igl
import os
import k3d

# Parameters to adjust

In [2]:
# Change contact range in your desire
Contac_parameter={
    "Contact_detected":0.04,  # contact detected
    "Good_contact":-0.015, 
    "Bad_contact":-0.05}    

# Change directory names
Dir_UPPER='UPPER_dir' # directory contains maxilla models. File names should follow a numbering format such as 1.stl, 2.stl, etc.
Dir_LOWER='LOWER_dir' # The directory contains mandibula models. File names should follow a numbering format such as 1.stl, 2.stl, etc.

# Main

In [13]:
Dir_UPPER_list=os.listdir(Dir_UPPER)
Dir_LOWER_list=os.listdir(Dir_LOWER)

for upper,lower in zip(Dir_UPPER_list,Dir_LOWER_list):
    Up=trimesh.load(os.path.join(Dir_UPPER,upper)) 
    Lo=trimesh.load(os.path.join(Dir_LOWER,lower))
    
    Lo_OG=copy.copy(Lo)
    Up_OG=copy.copy(Up)
    
    #---- Initial segmentation
    SDsDF_uper, _, _ = igl.signed_distance(Up.vertices.astype(float), Lo.vertices.astype(float), Lo.faces.astype(int))
    SDsDF_uper = np.where(SDsDF_uper < Contac_parameter["Contact_detected"])

    #----Sample initial
    Up.update_faces(np.unique(Up.vertex_faces[SDsDF_uper[0]])[1:])
    Up.remove_unreferenced_vertices()
    
    initial_sample=abs(Contac_parameter["Contact_detected"])/2
    select_sample=(initial_sample)**2
    Sample_size=int(Up.area/(select_sample))
    
    Up_sample,face_index=trimesh.sample.sample_surface_even(Up,Sample_size)
    Up_sampes_normals=Up.face_normals[face_index]
    
    #----Refine found samples
    SDsDF_uper, _, _ = igl.signed_distance(Up_sample.astype(float), Lo.vertices.astype(float), Lo.faces.astype(int))
    Up_sample=Up_sample[np.where(SDsDF_uper <= Contac_parameter["Contact_detected"]),:][0]
    Up_sampes_normals=Up_sampes_normals[np.where(SDsDF_uper <= Contac_parameter["Contact_detected"]),:][0]
    
    SDsDF_uper=SDsDF_uper[np.where(SDsDF_uper <= Contac_parameter["Contact_detected"])]
    
    SDsDF_uper_OG=copy.deepcopy(SDsDF_uper)
    
    SDsDF_uper_colors = np.zeros(SDsDF_uper.shape[0], dtype=np.int8)
    SDsDF_uper_colors[(SDsDF_uper >= Contac_parameter["Bad_contact"]) & (SDsDF_uper <  Contac_parameter["Good_contact"])] = 1  
    SDsDF_uper_colors[SDsDF_uper >= Contac_parameter["Good_contact"]] = 2   
    
    # ---- Stats
    Area_all=((Up.area*Up_sample.shape[0])/Sample_size).round(0)
    Threshold_Good_contac=((Up.area*np.where(SDsDF_uper > Contac_parameter["Good_contact"])[0].shape[0])/Sample_size).round(0)
    Threshold_Bad_contac=((Up.area*np.where(SDsDF_uper < Contac_parameter["Bad_contact"])[0].shape[0])/Sample_size).round(0)
    
    # colors
    palette = {0: 0xFF0000, 1: 0xFFA500, 2: 0x00FF00}
    
    labels = SDsDF_uper_colors.astype(int)
    colors = np.array([palette[val] for val in labels], dtype=np.uint32)

    print('----'+upper[0].split('.')[0]+' file ----')
    print(f"All contact area: {Area_all} mm²")
    print(f"Good contact (green): {Threshold_Good_contac} mm²")
    print(f"Bad contact (red): {Threshold_Bad_contac} mm²")
    print(f"Intermediate contact (orange): {(Area_all-Threshold_Good_contac-Threshold_Bad_contac).round(0)} mm²")
    
    # --- Plot results---
    plot = k3d.plot(grid_visible=False, background_color=0xFFFFFF)
    k3d_mesh = k3d.mesh(Up_OG.vertices, Up_OG.faces, color=0xCCCCCC, wireframe=False, opacity=1.0, flat_shading=False)
    k3d_points = k3d.points(Up_sample, point_size=0.05, colors=colors)
    plot += k3d_mesh
    plot += k3d_points
    plot.display()

----0 file ----
All contact area: 187.0 mm²
Good contact (green): 33.0 mm²
Bad contact (red): 123.0 mm²
Intermediate contact (orange): 31.0 mm²


/opt/venv/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/opt/venv/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()